In [ ]:
!pip install Datasets evaluate peft

In [ ]:
from google.colab import userdata
Llama_token = userdata.get('Llama_token')

In [ ]:
import torch
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import LlamaTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, LoraModel
from transformers import LlamaForSequenceClassification
from sklearn.model_selection import train_test_split
import evaluate

In [ ]:
file_path = "training_data.csv"
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,OriginalTweet,Sentiment
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,1,advice Talk to your neighbours family to excha...,Positive
2,2,Coronavirus Australia: Woolworths to give elde...,Positive
3,3,My food stock is not the only one which is emp...,Positive
4,4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
# Preprocess labels
label_mapping = {"Extremely Negative": 0, "Negative": 1, "Neutral": 2, "Positive": 3, "Extremely Positive": 4}
df["label"] = df["Sentiment"].map(label_mapping)

In [ ]:
df.shape

(41157, 4)

In [ ]:
# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(df["OriginalTweet"], df["label"], test_size=0.2, random_state=42)
# Convert to Hugging Face dataset
train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "label": train_labels.tolist()})
test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "label": test_labels.tolist()})
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})

In [ ]:
from huggingface_hub import login

# Replace 'your_huggingface_token_here' with your actual token
login(Llama_token)


In [ ]:
# Load tokenizer and model
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
model = LlamaForSequenceClassification.from_pretrained(model_name, num_labels=5)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Apply LoRA fine-tuning
lora_config = LoraConfig(
    r=8,  # Rank of the LoRA update matrices
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

In [ ]:
# Tokenize dataset
tokenized_datasets = datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/32925 [00:00<?, ? examples/s]

Map:   0%|          | 0/8232 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # LoRA allows a larger batch size
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Metric function
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-16-f1c42c74dae2>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: Bad StatusOr access: RESOURCE_EXHAUSTED: Error allocating device buffer: Attempting to allocate 64.00M. That was not possible. There are 16.70M free.; (0x0x0_HBM0)

In [ ]:
# Train the model
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

In [ ]:
trainer.save_model("llama2-lora-finetuned-text-classification")

In [ ]:
# !pip install evaluate datasets
# !pip install optimum
# !pip install auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 103.5 MB/s eta 0:00:00


# GPTQ

In [ ]:
import torch
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import LlamaTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, LoraModel
from transformers import LlamaForSequenceClassification
from sklearn.model_selection import train_test_split
import evaluate
from transformers import GPTQConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
from google.colab import userdata
Llama_token = userdata.get('Llama_token')

In [ ]:
file_path = "training_data.csv"
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,OriginalTweet,Sentiment
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,1,advice Talk to your neighbours family to excha...,Positive
2,2,Coronavirus Australia: Woolworths to give elde...,Positive
3,3,My food stock is not the only one which is emp...,Positive
4,4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
# # Preprocess labels
# label_mapping = {"Extremely Negative": 0, "Negative": 1, "Neutral": 2, "Positive": 3, "Extremely Positive": 4}
# df["label"] = df["Sentiment"].map(label_mapping)

In [ ]:
label = ['Neutral','Positive','Extremely Positive','Extremely Negative','Negative']
custom_data = []
for i in range(12):
    if df.iloc[i]['Sentiment'] in label:
        ex1 = df.iloc[i]['OriginalTweet']
        custom_data.append(ex1)
        label.remove(df.iloc[i]['Sentiment'])

In [ ]:
# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(df["OriginalTweet"], df["Sentiment"], test_size=0.2, random_state=42)
# Convert to Hugging Face dataset
train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "Sentiment": train_labels.tolist()})
test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "Sentiment": test_labels.tolist()})
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})

In [ ]:
model_path = "meta-llama/Llama-2-7b-hf"

custom_data = custom_data
token = Llama_token

tokenizer = AutoTokenizer.from_pretrained(model_path, token = token)

config = GPTQConfig(
    bits=4,
    dataset=custom_data,
    tokenizer=tokenizer,
    group_size=64,
    desc_false=False,
    model_seq_len=512,
)

model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=config,
        device_map="auto",
        token=token
)

model.save_pretrained("llama2-gptq-finetuned")
tokenizer.save_pretrained("llama2-gptq-finetuned-token")

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/7 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('llama2-gptq-finetuned-token/tokenizer_config.json',
 'llama2-gptq-finetuned-token/special_tokens_map.json',
 'llama2-gptq-finetuned-token/tokenizer.model',
 'llama2-gptq-finetuned-token/added_tokens.json',
 'llama2-gptq-finetuned-token/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "llama2-gptq-finetuned"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(f"{model_path}-token")

# # Load Quantized Model
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     device_map="auto",
#     torch_dtype=torch.float16
# )
from transformers import AutoModelForSequenceClassification

num_labels = 5  # Number of unique labels in the dataset

model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=num_labels,  # Important for classification
    device_map="auto",
    torch_dtype=torch.float16
)


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at llama2-gptq-finetuned and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd
from datasets import Dataset

# Load CSV data
train_df = pd.read_csv("training_data.csv")  # Replace with your actual file path
test_df = pd.read_csv("testing_data.csv")

df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df = df_shuffled.iloc[:int(0.25 * len(df_shuffled))]

# Preprocess labels
label_mapping = {"Extremely Negative": 0, "Negative": 1, "Neutral": 2, "Positive": 3, "Extremely Positive": 4}
train_df["label"] = train_df["Sentiment"].map(label_mapping)
# Preprocess labels
label_mapping = {"Extremely Negative": 0, "Negative": 1, "Neutral": 2, "Positive": 3, "Extremely Positive": 4}
test_df["label"] = test_df["Sentiment"].map(label_mapping)



# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Show a sample
train_dataset[0]

<ipython-input-55-cb0f93bad3c1>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["label"] = train_df["Sentiment"].map(label_mapping)


{'Unnamed: 0': 31089,
 'OriginalTweet': ' Without the there would not be any problem whatsoever People are getting worried about the supply chain   Prices for key food staples are starting to soar in some parts of the world via',
 'Sentiment': 'Neutral',
 'label': 2}

In [ ]:
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA config
lora_config = LoraConfig(
    r=8,  # Low-rank adaptation dimension
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,  # Sequence classification task
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,214,784 || all params: 135,573,504 || trainable%: 3.1089


In [ ]:
# Set pad_token to eos_token (or define a new one)
tokenizer.pad_token = tokenizer.eos_token  # Use EOS as padding
# # OR add a new pad token explicitly
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Resize model embeddings to account for new special tokens
model.resize_token_embeddings(len(tokenizer))

# Tokenization function
def preprocess_function(examples):
    tokenized_inputs = tokenizer(
        examples["OriginalTweet"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokenized_inputs["labels"] = examples["label"]  # Add labels
    return tokenized_inputs

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Convert labels to PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/10289 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Update from 'evaluation_strategy'
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True
)


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset
# )

# trainer.train()

In [ ]:
import torch
import torch.nn.functional as F
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Use CustomTrainer instead of Trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.000000,nan


TrainOutput(global_step=10289, training_loss=0.0004766069194042181, metrics={'train_runtime': 3603.7558, 'train_samples_per_second': 2.855, 'train_steps_per_second': 2.855, 'total_flos': 142282674143232.0, 'train_loss': 0.0004766069194042181, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': nan,
 'eval_runtime': 728.0693,
 'eval_samples_per_second': 5.217,
 'eval_steps_per_second': 5.217,
 'epoch': 1.0}

In [ ]:
model.save_pretrained("/content/drive/MyDrive/HackCU-11/llama2-gptq-lora-classifier")
tokenizer.save_pretrained("/content/drive/MyDrive/HackCU-11/llama2-gptq-lora-classifier")

('/content/drive/MyDrive/HackCU-11/llama2-gptq-lora-classifier/tokenizer_config.json',
 '/content/drive/MyDrive/HackCU-11/llama2-gptq-lora-classifier/special_tokens_map.json',
 '/content/drive/MyDrive/HackCU-11/llama2-gptq-lora-classifier/tokenizer.model',
 '/content/drive/MyDrive/HackCU-11/llama2-gptq-lora-classifier/added_tokens.json',
 '/content/drive/MyDrive/HackCU-11/llama2-gptq-lora-classifier/tokenizer.json')

# Without GPTQ


In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
model = LlamaForSequenceClassification.from_pretrained(model_name, num_labels=5)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import pandas as pd
from datasets import Dataset

# Load CSV data
train_df = pd.read_csv("training_data.csv")  # Replace with your actual file path
test_df = pd.read_csv("testing_data.csv")

df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df = df_shuffled.iloc[:int(0.25 * len(df_shuffled))]

# Preprocess labels
label_mapping = {"Extremely Negative": 0, "Negative": 1, "Neutral": 2, "Positive": 3, "Extremely Positive": 4}
train_df["label"] = train_df["Sentiment"].map(label_mapping)
# Preprocess labels
label_mapping = {"Extremely Negative": 0, "Negative": 1, "Neutral": 2, "Positive": 3, "Extremely Positive": 4}
test_df["label"] = test_df["Sentiment"].map(label_mapping)



# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Show a sample
train_dataset[0]

<ipython-input-67-cb0f93bad3c1>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["label"] = train_df["Sentiment"].map(label_mapping)


{'Unnamed: 0': 31089,
 'OriginalTweet': ' Without the there would not be any problem whatsoever People are getting worried about the supply chain   Prices for key food staples are starting to soar in some parts of the world via',
 'Sentiment': 'Neutral',
 'label': 2}

In [ ]:
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA config
lora_config = LoraConfig(
    r=8,  # Low-rank adaptation dimension
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,  # Sequence classification task
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,214,784 || all params: 6,611,578,880 || trainable%: 0.0637


In [ ]:
# Set pad_token to eos_token (or define a new one)
tokenizer.pad_token = tokenizer.eos_token  # Use EOS as padding
# # OR add a new pad token explicitly
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Resize model embeddings to account for new special tokens
model.resize_token_embeddings(len(tokenizer))

# Tokenization function
def preprocess_function(examples):
    tokenized_inputs = tokenizer(
        examples["OriginalTweet"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokenized_inputs["labels"] = examples["label"]  # Add labels
    return tokenized_inputs

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Convert labels to PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/10289 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Update from 'evaluation_strategy'
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True
)


In [ ]:
import torch
import torch.nn.functional as F
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

# Use CustomTrainer instead of Trainer
trainer_llama = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer_llama.train()

Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67c495d4-55201ad3465b41f556f5bc97;b9f5d1eb-c568-47aa-8085-f2b514fded0a)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-hf.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-hf - will assume that the vocabulary was not modified.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.547700,1.548184


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67c49c24-7e42d82240e48a2335e63ce5;d2aec0e4-5017-4447-9ee7-c4f31077bffd)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-hf.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-hf - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=10289, training_loss=1.654056148888685, metrics={'train_runtime': 10422.1983, 'train_samples_per_second': 0.987, 'train_steps_per_second': 0.987, 'total_flos': 2.0483461720571904e+17, 'train_loss': 1.654056148888685, 'epoch': 1.0})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/HackCU-11/llama2-lora-classifier")
tokenizer.save_pretrained("/content/drive/MyDrive/HackCU-11/llama2-lora-classifier")

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-67c49c6a-2deaa9f342fbc9091700da05;71596140-723b-479f-bfcc-1206c025e2be)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-hf is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-hf.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-hf - will assume that the vocabulary was not modified.
  warnings.warn(


('/content/drive/MyDrive/HackCU-11/llama2-lora-classifier/tokenizer_config.json',
 '/content/drive/MyDrive/HackCU-11/llama2-lora-classifier/special_tokens_map.json',
 '/content/drive/MyDrive/HackCU-11/llama2-lora-classifier/tokenizer.model',
 '/content/drive/MyDrive/HackCU-11/llama2-lora-classifier/added_tokens.json')

# Quantization of the Fine-tuned Llama

In [ ]:
import torch
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import LlamaTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, LoraModel
from transformers import LlamaForSequenceClassification
from sklearn.model_selection import train_test_split
import evaluate
from transformers import GPTQConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
file_path = "training_data.csv"
df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,OriginalTweet,Sentiment
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,1,advice Talk to your neighbours family to excha...,Positive
2,2,Coronavirus Australia: Woolworths to give elde...,Positive
3,3,My food stock is not the only one which is emp...,Positive
4,4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
label = ['Neutral','Positive','Extremely Positive','Extremely Negative','Negative']
custom_data = []
for i in range(12):
    if df.iloc[i]['Sentiment'] in label:
        ex1 = df.iloc[i]['OriginalTweet']
        custom_data.append(ex1)
        label.remove(df.iloc[i]['Sentiment'])

In [ ]:
model_path = "/content/drive/MyDrive/HackCU-11/llama2-lora-classifier"

custom_data = custom_data
# token = Llama_token

tokenizer = AutoTokenizer.from_pretrained(model_path)

config = GPTQConfig(
    bits=4,
    dataset=custom_data,
    tokenizer=tokenizer,
    group_size=64,
    desc_false=False,
    model_seq_len=512,
)

model = AutoModelForSequenceClassification.from_pretrained(
        model_path,
        num_labels=5,
        quantization_config=config,
        token = token,
        device_map="auto"
)

In [ ]:
model.summary()

In [ ]:
model.save_pretrained("/content/drive/MyDrive/HackCU-11/fine-tuned-llama2-gptq-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/HackCU-11/fine-tuned-llama2-gptq-finetuned-token")